# FIT5202: Assignment 2B
## Ayush Sharma
## 30823293


## Task3 

# Importing libraries

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0 pyspark-shell'

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import SparkConf
from pyspark.sql.functions import explode
from pyspark.sql.functions import udf
import datetime 
import time
from pyspark.ml import PipelineModel
import datetime as dt
from  pyspark.sql.functions import abs
from pyspark.sql.functions import lit

### Task 3.1

In [2]:
# local[*]: run Spark in local mode with as many working processors as logical cores on your machine
# If we want Spark to run locally with 'k' worker threads, we can specify as "local[k]".
master = "local[2]"
# The `appName` field is a name to be shown on the Spark cluster UI page
app_name = "Spark Streaming Data"
# Setup configuration parameters for Spark
spark_conf = SparkConf().setMaster(master).setAppName(app_name).set("spark.sql.session.timeZone", "UTC")

# Method 1: Using SparkSession
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')


## Task 3.2
Ingest the streaming data into Spark Streaming for both process and memory activities

In [3]:
# specify topic
topic = "NewLinuxMemoryStream2B"
#loading streaming data
memoryDf = spark \
           .readStream \
           .format("kafka") \
           .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
           .option("subscribe", topic) \
           .load()

In [4]:
# specify topic
topic = "NewLinuxProcessStream2B"


processDf = spark \
           .readStream \
           .format("kafka") \
           .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
           .option("subscribe", topic) \
           .load()

In [5]:
memoryDf = memoryDf.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [6]:
processDf = processDf.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [7]:
# Schema for Memory Stream
memorySchema = ArrayType(StructType([    
  StructField("sequence", IntegerType(), True),
  StructField("machine", IntegerType(), True),
  StructField("PID", IntegerType(), True),
  StructField("MINFLT", StringType(), True),
  StructField("MAJFLT", StringType(), True),
  StructField("VSTEXT", StringType(), True),
  StructField("VSIZE", FloatType(), True),
  StructField("RSIZE", StringType(), True),
  StructField("VGROW", StringType(), True),
  StructField("RGROW", StringType(), True),
  StructField("MEM", FloatType(), True),
  StructField("CMD", StringType(), True),
  StructField("ts", IntegerType(), True)   
]))

In [8]:
# Schema for Process Stream

processSchema = ArrayType(StructType([
    StructField("sequence", IntegerType(), True),
    StructField("machine", IntegerType(), True),
    StructField("PID", IntegerType(), True),
    StructField("TRUN", IntegerType(), True),
    StructField("TSLPI", IntegerType(), True),
    StructField("TSLPU", IntegerType(), True),
    StructField("POLI", StringType(), True),
    StructField("NICE", FloatType(), True),
    StructField("PRI", FloatType(), True),
    StructField("RTPR", FloatType(), True),
    StructField("CPUNR", FloatType(), True),
    StructField("Status", StringType(), True),
    StructField("EXC", IntegerType(), True),
    StructField("State", StringType(), True),
    StructField("CPU", FloatType(), True),
    StructField("CMD", StringType(), True),
    StructField("ts", IntegerType(), True),
]))

In [9]:
memoryDf = memoryDf.select(F.from_json(F.col("value").cast("string"),memorySchema)\
                           .alias('parsed_value'))

In [10]:
processDf = processDf.select(F.from_json(F.col("value").cast("string"),processSchema)\
                             .alias('parsed_value'))

In [11]:
memoryDf = memoryDf.select(F.explode(F.col("parsed_value"))\
                           .alias('unnested_value'))

In [12]:
processDf = processDf.select(F.explode(F.col("parsed_value"))\
                             .alias('unnested_value'))

In [13]:
#New formated schema for memory
newMemoryDf = memoryDf.select(
                    F.col("unnested_value.sequence").alias("Sequence"),
                    F.col("unnested_value.machine").alias("Machine"),
                    F.col("unnested_value.PID").alias("PID"),
                    F.col("unnested_value.MINFLT").alias("MINFLT"),
                    F.col("unnested_value.MAJFLT").alias("MAJFLT"),
                    F.col("unnested_value.VSTEXT").alias("VSTEXT"),
                    F.col("unnested_value.VSIZE").alias("VSIZE"),
                    F.col("unnested_value.RSIZE").alias("RSIZE"),
                    F.col("unnested_value.VGROW").alias("VGROW"),
                    F.col("unnested_value.RGROW").alias("RGROW"),
                    F.col("unnested_value.MEM").alias("MEM"),
                    F.col("unnested_value.CMD").alias("CMD"),
                    F.col("unnested_value.ts").alias("ts")                    
                )

In [14]:
#New formated schema for memory
newProcessDf = processDf.select(
                    F.col("unnested_value.sequence").alias("Sequence"),
                    F.col("unnested_value.machine").alias("Machine"),
                    F.col("unnested_value.PID").alias("PID"),
                    F.col("unnested_value.TRUN").alias("TRUN"),
                    F.col("unnested_value.TSLPI").alias("TSLPI"),
                    F.col("unnested_value.TSLPU").alias("TSLPU"),
                    F.col("unnested_value.POLI").alias("POLI"),
                    F.col("unnested_value.NICE").alias("NICE"),
                    F.col("unnested_value.PRI").alias("PRI"),
                    F.col("unnested_value.RTPR").alias("RTPR"),
                    F.col("unnested_value.CPUNR").alias("CPUNR"),
                    F.col("unnested_value.Status").alias("Status"),
                    F.col("unnested_value.EXC").alias("EXC"),
                    F.col("unnested_value.State").alias("State"),
                    F.col("unnested_value.CPU").alias("CPU"),
                    F.col("unnested_value.CMD").alias("CMD"),
                    F.col("unnested_value.ts").alias("ts")
                )

## Task 3.3

Reformating the Streaming Data from memory and process


In [15]:

#MINFLT replace and type change
replaceK = udf(lambda x: float(x.replace("K", ""))*1000 if "K" in x else float(x), FloatType())
newMemoryDf = newMemoryDf.withColumn("MINFLT", replaceK("MINFLT"))

#MAJFLT replace and type change
replaceM = udf(lambda x: float(x.replace("M", ""))*1000000 if "M" in x else float(x), FloatType())
newMemoryDf = newMemoryDf.withColumn("MAJFLT", replaceM("MAJFLT"))

#VSTEXT replace and type change
replaceK = udf(lambda x: float(x.replace("K", ""))*1000 if "K" in x else float(x), FloatType())
newMemoryDf= newMemoryDf.withColumn("VSTEXT", replaceK("VSTEXT"))


#RSIZE replace and type change
replaceM = udf(lambda x: str(float(x.replace("M", ""))*1000000) if "M" in x else x, StringType())
newMemoryDf = newMemoryDf.withColumn("RSIZE", replaceM("RSIZE"))

replaceK = udf(lambda y: str(float(y.replace("K", ""))*1000) if "K" in y else y, StringType())
newMemoryDf = newMemoryDf.withColumn("RSIZE", replaceK("RSIZE"))

newMemoryDf = newMemoryDf.withColumn("RSIZE", newMemoryDf["RSIZE"].cast(FloatType()))

#VGROW replace and type change
replaceSPC = udf(lambda x: str(float(x.replace(" ", ""))) if " " in x else x, StringType())
newMemoryDf = newMemoryDf.withColumn("VGROW", replaceSPC("VGROW"))

replaceK = udf(lambda y: str(float(y.replace("K", ""))*1000) if "K" in y else y, StringType())
newMemoryDf = newMemoryDf.withColumn("VGROW", replaceK("VGROW"))

newMemoryDf = newMemoryDf.withColumn("VGROW", newMemoryDf["VGROW"].cast(FloatType()))

#RGROW replace and type change
replaceK = udf(lambda x: float(x.replace("K", ""))*1000 if "K" in x else float(x), FloatType())
newMemoryDf = newMemoryDf.withColumn("RGROW", replaceK("RGROW"))

In [16]:
#UDF for mapping PRI and NICE
newPRI_Nice = udf(lambda x: x - 120  , FloatType())
newProcessDf = newProcessDf.withColumn("NICE", newPRI_Nice('PRI'))

# Task 3.4

Introducting new column by merging CMD and PID and a new event_time using ts column

In [17]:
# UDF for New column CMD_PRI from CMD and PRI for process
newCMD_PID = udf(lambda x: str(x[0] + "_" + x[1]), StringType())
newProcessDf = newProcessDf.withColumn("CMD_PID", newCMD_PID(array('CMD', 'PID')))

In [18]:
# UDF for New column CMD_PRI from CMD and PRI for memory
newMemoryDf = newMemoryDf.withColumn("CMD_PID", newCMD_PID(array('CMD', 'PID')))

In [19]:
# Adding event_time using the ts columnn
eventTime = udf(lambda x: datetime.datetime.strptime(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x)),'%Y-%m-%d %H:%M:%S'),TimestampType())
# For process
newProcessDf = newProcessDf.withColumn("event_time", eventTime('ts'))
# For memory
newMemoryDf = newMemoryDf.withColumn("event_time", eventTime('ts'))

# Task 3.5

In [20]:
# creating and writing into parquet files for memory stream
query_fileSink_process = newProcessDf.writeStream.format("parquet")\
        .outputMode("append")\
        .option("path", "process.parquet")\
        .option("checkpointLocation", "process.parquet/checkpoint")\
        .start()

In [21]:
#Stop the file_sink query
query_fileSink_process.stop()

In [22]:
# creating and writing into parquet files for process stream
query_fileSink_memory = newMemoryDf.writeStream.format("parquet")\
        .outputMode("append")\
        .option("path", "memory.parquet")\
        .option("checkpointLocation", "memory.parquet/checkpoint")\
        .start()

In [23]:
#Stop the file_sink query
query_fileSink_memory.stop()

# 3.6 ML Models

In [24]:
#Load process pipeline 
process_pipelineModel = PipelineModel.load('process_pipeline_model')

In [25]:
#Loading memory pipeline 
memory_pipelineModel=PipelineModel.load('memory_pipeline_model')

In [26]:
#Transform Process Model
Process_predict=process_pipelineModel.transform(newProcessDf)

In [27]:
#Transform Process Model
memory_predict = memory_pipelineModel.transform(newMemoryDf)

## Task 3.7 A

In [28]:
# Data filtered based on if it is attacked i.e. =1 for process
process_predictAttack = Process_predict.filter(Process_predict.prediction == 1)

In [29]:
# Data filtered based on if it is attacked i.e. =1 for memory
memory_predictAttack = memory_predict.filter(memory_predict.prediction == 1)

In [30]:
# creating event group by based on event_time
windowedCounts = process_predictAttack \
    .withWatermark("event_time", "120 seconds") \
    .groupBy(window(process_predictAttack.event_time, "120 seconds"),process_predictAttack.Machine)\
    .agg(F.count("*").alias("total_Attack"))\
    .select("window","total_Attack","Machine")

In [31]:
# running and streaming query
processAttackquery = windowedCounts \
    .writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("queryProcessPrediction") \
    .trigger(processingTime='5 seconds') \
    .option("truncate","false")\
    .start()

In [32]:
#Stop the query
processAttackquery.stop()

In [33]:
# query for attack based on event_time and Machine
windowedCounts1 = memory_predictAttack \
    .withWatermark("event_time", "120 seconds") \
    .groupBy(window(memory_predictAttack.event_time, "120 seconds"),memory_predictAttack.Machine)\
    .agg(F.count("*").alias("total_attack"))\
    .select("window","total_attack","Machine")

In [34]:
processAttackqueryNew = windowedCounts1 \
    .writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("queryMemoryPrediction") \
    .trigger(processingTime='5 seconds') \
    .option("truncate","false")\
    .start()

In [35]:
# stopping the query
processAttackqueryNew.stop()

# Task 3.7 B

In [36]:
#updating column names in memory
newMemoryPrediction = memory_predictAttack.withColumnRenamed("ts","ts_memory").withColumnRenamed("CMD_PID","CMD_PID_memory")\
                    .withColumnRenamed("prediction","prediction_memory").withColumnRenamed("machine","machine_memory").withColumnRenamed("event_time","event_time_memory")

In [37]:
#updating column names in memory
newProcessPrediction = process_predictAttack.withColumnRenamed("ts","ts_process").withColumnRenamed("CMD_PID","CMD_PID_process")\
                    .withColumnRenamed("prediction","prediction_process").withColumnRenamed("machine","machine_process").withColumnRenamed("event_time","event_time_process")

In [38]:
# merging the process and memory dataframe
newFinalData = newMemoryPrediction.join(newProcessPrediction)

In [39]:
#Filterring for event times in process and memory is less than 30 seconds
# filtering memory and process abased on event_time
newFinalData1 = newFinalData.filter(abs(newFinalData.ts_process - newFinalData.ts_memory) <= 30)

In [40]:
#new column Processing_Time
newFinalData2 = newFinalData1.withColumn("processing_time",lit(dt.datetime.utcnow()))

In [41]:
query_sink_merged = newFinalData2.writeStream.format("parquet")\
        .outputMode("append")\
        .option("path", "process_memory_attack.parquet")\
        .option("checkpointLocation", "process_memory_attack/checkpoint")\
        .start()

In [42]:
query_sink_merged.stop()